In [3]:
import json
import os
import numpy as np
from keras.models import Model,load_model
from keras.layers import Input, LSTM, Dense,Activation,Embedding,merge
from keras.layers.core import Lambda
from keras.layers import concatenate,Reshape,Concatenate,RepeatVector
import numpy as np
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from nltk.translate.bleu_score import sentence_bleu
from keras.callbacks import EarlyStopping
import random
from PIL import Image
from matplotlib import pyplot as plt
import pickle
from keras import backend as K
import math

Using TensorFlow backend.


In [2]:
coco_json='./data/dataset_coco.json'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
KTF.set_session(session)

In [3]:
with open(coco_json) as file:
    lines=file.readlines()
data=json.loads(lines[0])['images']

In [7]:
train_x_file=[]
train_y_seq=[]
vali_x_file=[]
vali_y_seq=[]
token={}
if os.path.exists('tokenv2.pkl'):
    with open('tokenv2.pkl','rb') as file:
        token=pickle.load(file)
else:
    token['\t']=1
    token['\n']=2

EOFError: Ran out of input

In [5]:
for d in data:
    if d['split']=='train':
        train_x_file.append('./coco/'+d['filepath']+'/'+d['filename'])
        seq=d['sentences'][0]['tokens']
        for s in seq:
            if s not in token:
                token[s]=len(token)+1
        train_y_seq.append(['\t']+seq+['\n'])
    if d['split']=='test':
        vali_x_file.append('./coco/'+d['filepath']+'/'+d['filename'])
        seq=d['sentences'][0]['tokens']
        for s in seq:
            if s not in token:
                token[s]=len(token)+1
        vali_y_seq.append(['\t']+seq+['\n'])

In [5]:
with open('tokenv2.pkl','wb') as file:
    pickle.dump(token,file)
    
num_tokens=len(token)
batch_size=1
seq_len=49+2 #49+'\t'+'\n'
vec_len=173056
print("Num of train:",len(train_x_file))
print("Num of vali:",len(vali_x_file))
print("Num of token:",num_tokens)
print("Len of vector:",vec_len)


NameError: name 'token' is not defined

In [7]:
def vec_reader(path):
    with open(path,'r') as f:
        line=f.readline()
        vector=np.asarray( list(map(float,line.split(",")[1].split(" "))) )
    return vector

In [8]:
def dataGenerator(x,y):
    while(1):
        decoder_input_data=[]
        decoder_target_data=[]
        encoder_input_data=[]
        for index in range(len(x)):
            for t,word in enumerate(y[index]):
                if t>0:
                    did=np.zeros(
                        (seq_len),dtype='float32'
                    )
                    dtd=np.zeros(
                        (1),dtype='float32'
                    )
                    
                    eid=vec_reader(x[index]+'.txt')
                    #eid=np.reshape(eid,(1,vec_len))
                    encoder_input_data.append(eid)
                    for tt in range(t):
                        did[tt]=float(token[y[index][tt]])
                    dtd[0]=float(token[y[index][t]])
                    decoder_input_data.append(did)
                    decoder_target_data.append(dtd)
                    if len(encoder_input_data)==batch_size:
                        yield ([np.array(encoder_input_data),np.array(decoder_input_data)],np.array(decoder_target_data))
                        decoder_input_data=[]
                        decoder_target_data=[]
                        encoder_input_data=[]

In [4]:
latent_dim=256
encode_input=Input(shape=(vec_len,))
encode_dense=Dense(latent_dim,activation='relu')(encode_input)
#encode_dense=Reshape((1,latent_dim))(encode_dense)
encode_dense=RepeatVector(1)(encode_dense)
decode_input=Input(shape=(seq_len,))
decode_emb=Embedding(num_tokens+1,latent_dim,input_length=(seq_len))(decode_input)
decoder_vec=concatenate([decode_emb,encode_dense],axis=1)
decoder=LSTM(latent_dim)(decoder_vec)
decoder=Dense(num_tokens,activation='softmax')(decoder)
model=Model([encode_input,decode_input],decoder)

NameError: name 'vec_len' is not defined

In [10]:
def ppx(y_true, y_pred):
     loss = K.sparse_categorical_crossentropy(y_true, y_pred)
     perplexity = K.cast(K.pow(math.e, K.mean(loss, axis=-1)), K.floatx())
     return perplexity

In [1]:
model.summary()
model.compile(loss=ppx, optimizer='adam',metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
model.fit_generator(
    dataGenerator(train_x_file,train_y_seq),
    #validation_data=dataGenerator(vali_x_file,vali_y_seq),
    steps_per_epoch=((len(train_x_file))//batch_size),
    #steps_per_epoch=10000,
    #validation_steps=1,
    epochs=200
    #callbacks=[cbes]
)

Epoch 1/200
  609/82783 [..............................] - ETA: 1:21:41 - loss: 1766.9033 - acc: 0.0000e+00